<a href="https://colab.research.google.com/github/Marwuko/blog_astro/blob/main/202412069.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from google.colab import drive
drive.mount('/content/drive')


file_path = "/content/drive/MyDrive/Group_prediction_by_Regressor-Dataset_(2).xlsx"
df = pd.read_excel(file_path)

print("Dataset Loaded Successfully")
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset Loaded Successfully


,Gender,Father Highest Academic Qualification,Mother Highest Academic Qualification,Father Profession,Mother Profession,Father average monthly income,Mother average monthly income,How many siblings do you have,District Currently you are living,PEC Result Overall GPA,...,JSC Overall GPA,JSC Bangla,JSC English,JSC Mathematics,JSC BGS,JSC ICT,JSC Religion,JSC Science,Group SSC,Overall GPA SSC
0,0,4,2,0,2,20000,0,3,0,5.0,...,5.00,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0,5.00
1,0,5,3,2,2,57000,0,1,0,5.0,...,5.00,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0,5.00
2,0,4,2,1,2,100000,80000,2,1,5.0,...,4.65,4.0,4.0,5.0,4.0,5.0,4.0,5.0,0,4.11
3,0,5,5,1,2,400000,100000,1,0,5.0,...,5.00,4.0,5.0,5.0,5.0,5.0,5.0,5.0,0,4.61
4,1,0,0,1,2,50000,0,3,0,5.0,...,4.30,4.0,4.0,5.0,5.0,4.0,4.0,5.0,0,4.50


In [2]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [3]:
def evaluate_model(name, model):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)

    return {
        "Model": name,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R2 Score": r2
    }

In [4]:
results = []

lr_model = LinearRegression()
results.append(evaluate_model("Linear Regression", lr_model))

rf_model = RandomForestRegressor(n_estimators=300, random_state=42)
results.append(evaluate_model("Random Forest Regressor", rf_model))

xgb_model = XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
results.append(evaluate_model("XGBoost Regressor", xgb_model))


In [5]:
results_df = pd.DataFrame(results)
results_df

,Model,MAE,MSE,RMSE,R2 Score
0,Linear Regression,0.246241,0.088473,0.297445,0.053868
1,Random Forest Regressor,0.204000,0.077208,0.277863,0.174341
2,XGBoost Regressor,0.205296,0.072838,0.269884,0.221077


from matplotlib import pyplot as plt
results_df['MAE'].plot(kind='hist', bins=20, title='MAE')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['MSE'].plot(kind='hist', bins=20, title='MSE')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['RMSE'].plot(kind='hist', bins=20, title='RMSE')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['R2 Score'].plot(kind='hist', bins=20, title='R2 Score')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='MSE', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='RMSE', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='R2 Score', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [6]:
best_model = results_df.loc[results_df["R2 Score"].idxmax()]
best_model

,2
Model,XGBoost Regressor
MAE,0.205296
MSE,0.072838
RMSE,0.269884
R2 Score,0.221077
